# calculate-CuSurface-COAds.ipynb

ase_toolboxを使用して、Cu表面へのCO吸着エネルギーを計算するサンプルです。

## 0. 事前準備

In [ ]:
%cd ~
import pfp_api_client
from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode
from pfcc_extras.visualize.view import view_ngl


estimator = Estimator(calc_mode=EstimatorCalcMode.CRYSTAL_U0)
calculator = ASECalculator(estimator)

print(f"pfp_api_client: {pfp_api_client.__version__}")


## 1. 構造の作成

### 1.1 Cu表面

In [ ]:
from ase.build import bulk, surface
from ase_toolbox.HandleAtoms import fix_layers

# Cu表面を作る (3x3x4 スラブ、下2層固定)
cu_bulk = bulk("Cu", "fcc")
cu_surface = surface(cu_bulk, (1, 1, 1), layers=4, vacuum=10.0) * (3, 3, 1)
cu_surface.center(axis=2)  # 原子の重心を中心に調整する

# fix_layersを使用して、下2層を固定する
cu_surface = fix_layers(cu_surface, 2)

view_ngl(cu_surface, representations=["ball+stick"])


### 1.2 CO分子

In [ ]:
from ase.build import molecule

co_molecule = molecule("CO")
co_molecule.center(axis=2)

view_ngl(co_molecule, representations=["ball+stick"])


### 1.3 COが吸着された構造

- `ase_toolbox.FindAtoms`内の関数を使用すると、吸着構造の作成が楽になる

In [ ]:
from ase.build import add_adsorbate
from ase_toolbox.FindAtoms import find_central_atom, separate_layers

# add_adsorbateすると上書きされてしまうので、コピーしておく
cu_co_adsorbed = cu_surface.copy()

# 一番上の層を取得
top_layer = separate_layers(cu_co_adsorbed)[-1]

# 中心の原子を取得
center_atom = find_central_atom(top_layer)

# 中心の原子に吸着させる
add_adsorbate(
    cu_co_adsorbed,
    co_molecule,
    height=3,
    position=center_atom.position[:2],
)

view_ngl(cu_co_adsorbed, representations=["ball+stick"])


## 2. 計算

- `ase_toolbox.Calculation`内の関数を使用すると、吸着エネルギーの計算が楽になる

In [ ]:
from ase_toolbox.Calculation import CAEInput, calculate_adsorption_energy

# ----------
# ---計算用の設定を用意する
# ----------
# ---吸着構造
adsorbed_input = CAEInput(structure=cu_co_adsorbed, calc_mode="solid")

# ---反応物
reactant_inputs = [
    CAEInput(structure=cu_surface, calc_mode="solid"),
    CAEInput(structure=co_molecule, calc_mode="solid"),
]

# ----------
# ---計算する
# ----------
# 吸着前・吸着後の構造を入力すると、自動で最適化し、吸着エネルギーを計算する
# 詳細な処理の流れは、ase_toolbox.Calculation.py内の `calculate_adsorption_energy` を参照すること
adsorption_energy = calculate_adsorption_energy(
    calculator_molecule=calculator,
    calculator_solid=calculator,
    adsorbed_structure_input=adsorbed_input,
    reactant_structures_input=reactant_inputs,
    opt_fmax=0.05,
    opt_maxsteps=3000,
    enable_logging=True,
    copy_atoms=False,  # これをFalseにすると、もとの入力の構造をコピーせず、上書きされる
)

print(f"吸着エネルギー: {adsorption_energy}")

view_ngl(cu_co_adsorbed)


## 3. そのほかのアイデア

### 3.1 すべての表面原子の中で、1つずつ吸着させてみる

In [ ]:
import pfp_api_client
from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode
from pfcc_extras.visualize.view import view_ngl
from ase.build import add_adsorbate, molecule, bulk, surface

from ase_toolbox.HandleAtoms import fix_layers
from ase_toolbox.Calculation import CAEInput, calculate_adsorption_energy
from ase_toolbox.FindAtoms import separate_layers

# ----------
# ---計算機を用意する
# ----------
estimator = Estimator(calc_mode=EstimatorCalcMode.CRYSTAL_U0)
calculator = ASECalculator(estimator)

print(f"pfp_api_client: {pfp_api_client.__version__}")


# ----------
# ---基本的な構造を作る
# ----------
# ---Cu表面
cu_bulk = bulk("Cu", "fcc")
cu_surface = surface(cu_bulk, (1, 1, 1), layers=4, vacuum=10.0) * (3, 3, 1)
cu_surface.center(axis=2)
cu_surface = fix_layers(cu_surface, 2)

# ---CO分子
co_molecule = molecule("CO")
co_molecule.center(axis=2)

# ---CAEInputを作る
reactant_inputs = [
    CAEInput(structure=cu_surface, calc_mode="solid"),
    CAEInput(structure=co_molecule, calc_mode="solid"),
]

# ----------
# ---吸着->計算
# ----------
adsorption_energy_labels = [""] * len(cu_surface)
top_layer = separate_layers(cu_surface)[-1]
for surface_atom in top_layer:
    # ---吸着構造を作る
    alloy_co_adsorbed = cu_surface.copy()
    add_adsorbate(
        alloy_co_adsorbed,
        co_molecule,
        height=3,
        position=surface_atom.position[:2],
    )

    # ---計算する
    adsorbed_input = CAEInput(structure=alloy_co_adsorbed, calc_mode="solid")

    adsorption_energy = calculate_adsorption_energy(
        calculator_molecule=calculator,
        calculator_solid=calculator,
        adsorbed_structure_input=adsorbed_input,
        reactant_structures_input=reactant_inputs,
        opt_fmax=0.05,
        opt_maxsteps=3000,
        enable_logging=True,
    )
    print(
        f"{'='*10}\n{surface_atom.index}番目の原子に吸着させたときの吸着エネルギー: {adsorption_energy}\n{'='*10}"
    )
    adsorption_energy_labels[surface_atom.index] = str(round(adsorption_energy,4))

v = view_ngl(cu_surface, representations=["ball+stick"])
v.view.add_label(
    color="black",
    labelType="text",
    labelText=adsorption_energy_labels,
    zOffset=1.0,
    attachment="middle_center",
    radius=0.5,
)
v


### 3.2 合金にしてみる

In [ ]:
import pfp_api_client
from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode
from pfcc_extras.visualize.view import view_ngl
from ase.build import add_adsorbate, molecule, bulk, surface

from ase_toolbox.HandleAtoms import fix_layers, substitute_elements
from ase_toolbox.Calculation import CAEInput, calculate_adsorption_energy
from ase_toolbox.FindAtoms import find_central_atom, separate_layers

# ----------
# ---計算機を用意する
# ----------
estimator = Estimator(calc_mode=EstimatorCalcMode.CRYSTAL_U0)
calculator = ASECalculator(estimator)

print(f"pfp_api_client: {pfp_api_client.__version__}")


# ----------
# ---構造を作る
# ----------
# ---Cu表面
cu_bulk = bulk("Cu", "fcc")
cu_surface = surface(cu_bulk, (1, 1, 1), layers=4, vacuum=10.0) * (3, 3, 1)
cu_surface.center(axis=2)
cu_surface = fix_layers(cu_surface, 2)

# ---Cuを合金化する
alloy_surface = substitute_elements(cu_surface, cu_surface, {"Cu": 0.8, "Au": 0.2})

# ---CO分子
co_molecule = molecule("CO")
co_molecule.center(axis=2)

# ---吸着構造
alloy_co_adsorbed = alloy_surface.copy()
top_layer = separate_layers(alloy_co_adsorbed)[-1]
center_atom = find_central_atom(top_layer)

# 中心の原子に吸着させる
add_adsorbate(
    alloy_co_adsorbed,
    co_molecule,
    height=3,
    position=center_atom.position[:2],
)

# ----------
# ---計算する
# ----------
# ---CAEInputを作る
adsorbed_input = CAEInput(structure=alloy_co_adsorbed, calc_mode="solid")
reactant_inputs = [
    CAEInput(structure=cu_surface, calc_mode="solid"),
    CAEInput(structure=co_molecule, calc_mode="solid"),
]
# ---計算する
adsorption_energy = calculate_adsorption_energy(
    calculator_molecule=calculator,
    calculator_solid=calculator,
    adsorbed_structure_input=adsorbed_input,
    reactant_structures_input=reactant_inputs,
    opt_fmax=0.05,
    opt_maxsteps=3000,
    enable_logging=True,
    copy_atoms=False,
)

print(f"吸着エネルギー: {adsorption_energy}")

view_ngl(alloy_co_adsorbed, representations=["ball+stick"])


### 3.3 表面だけ原子を変えてみる

In [ ]:
import pfp_api_client
from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode
from pfcc_extras.visualize.view import view_ngl
from ase.build import add_adsorbate, molecule, bulk, surface

from ase_toolbox.HandleAtoms import fix_layers, substitute_elements
from ase_toolbox.Calculation import CAEInput, calculate_adsorption_energy
from ase_toolbox.FindAtoms import find_central_atom, separate_layers

# ----------
# ---計算機を用意する
# ----------
estimator = Estimator(calc_mode=EstimatorCalcMode.CRYSTAL_U0)
calculator = ASECalculator(estimator)

print(f"pfp_api_client: {pfp_api_client.__version__}")


# ----------
# ---構造を作る
# ----------
# ---Cu表面
cu_bulk = bulk("Cu", "fcc")
cu_surface = surface(cu_bulk, (1, 1, 1), layers=4, vacuum=10.0) * (3, 3, 1)
cu_surface.center(axis=2)
cu_surface = fix_layers(cu_surface, 2)

# ---Cuの表面2層をAuにする
layers = separate_layers(cu_surface)
alloy_surface = substitute_elements(cu_surface, layers[-1], "Au")
alloy_surface = substitute_elements(alloy_surface, layers[-2], "Au")

# ---CO分子
co_molecule = molecule("CO")
co_molecule.center(axis=2)

# ---吸着構造
alloy_co_adsorbed = alloy_surface.copy()
top_layer = separate_layers(alloy_co_adsorbed)[-1]
center_atom = find_central_atom(top_layer)

# 中心の原子に吸着させる
add_adsorbate(
    alloy_co_adsorbed,
    co_molecule,
    height=3,
    position=center_atom.position[:2],
)

# ----------
# ---計算する
# ----------
# ---CAEInputを作る
adsorbed_input = CAEInput(structure=alloy_co_adsorbed, calc_mode="solid")
reactant_inputs = [
    CAEInput(structure=cu_surface, calc_mode="solid"),
    CAEInput(structure=co_molecule, calc_mode="solid"),
]
# ---計算する
adsorption_energy = calculate_adsorption_energy(
    calculator_molecule=calculator,
    calculator_solid=calculator,
    adsorbed_structure_input=adsorbed_input,
    reactant_structures_input=reactant_inputs,
    opt_fmax=0.05,
    opt_maxsteps=3000,
    enable_logging=True,
    copy_atoms=False,
)

print(f"吸着エネルギー: {adsorption_energy}")

view_ngl(alloy_co_adsorbed, representations=["ball+stick"])
